In [25]:
import os
import json

# Assume this is the directory containing your JSON files
testcaseDir = 'annotations/'
testOutput = 'testoutput/'
# List all JSON files in the directory
json_files = [pos_json for pos_json in os.listdir(testcaseDir) if pos_json.endswith('.json')]

 # Define the function to traverse and extract data
def accessJSON(data):
    for key, value in data.items():
        if isinstance(value, (str, int, float, bool)):
            stack.append(key)
            stack.append(value)
        elif isinstance(value, dict):
            accessJSON(value)
        elif isinstance(value, list):
            for val in value:
                if isinstance(val, dict):
                    accessJSON(val)
                    
# Process each JSON file
for index, js in enumerate(json_files):
    with open(os.path.join(testcaseDir, js)) as json_file:
        s = json_file.read()
        s = s.replace('\'', '\"')
        data = json.loads(s)  # Load JSON data
    fileName = (os.path.splitext(js)[0])

    print(f"Processing file: {js}")
    print("Loaded JSON data:")
    print(data)

    # Initialize stack to hold keys and values
    stack = []

    # Call the function to extract keys and values
    accessJSON(data)

    # Print the extracted keys and values
    print("Extracted keys and values:")
    print(stack)
    # Now use the previous stack to separate keys and values
    keys, values = seperatekeyvalues(stack)

    # Find the number of rows and columns
    rows, columns = findingrowscolumns(keys)
    
    # Define the output CSV file name
    fileDir = os.path.join(testOutput, fileName+".csv")
    # Write the data to the CSV file
    writingintoFile(fileDir, rows, columns, keys, values)

# Print a confirmation message
    print(f"Data has been successfully written to {outputFileName}")
# Print the results
    print("Number of rows:", rows)
    print("Number of columns:", columns)
    print("Keys:", keys)
    print("Values:", values)

Processing file: input7.json
Loaded JSON data:
{'persons': [{'firstname': 'John', 'lastname': 'Doe', 'lang': 'en', 'num': 3, 'dt': 'xsd:integer'}, {'firstname': 'Jane', 'lastname': 'Smith', 'lang': 'fr', 'num': 3.14, 'dt': 'xsd:decimal'}]}
Extracted keys and values:
['firstname', 'John', 'lastname', 'Doe', 'lang', 'en', 'num', 3, 'dt', 'xsd:integer', 'firstname', 'Jane', 'lastname', 'Smith', 'lang', 'fr', 'num', 3.14, 'dt', 'xsd:decimal']
Data has been successfully written to output.csv
Number of rows: 2
Number of columns: 5
Keys: ['firstname', 'lastname', 'lang', 'num', 'dt', 'firstname', 'lastname', 'lang', 'num', 'dt']
Values: ['John', 'Doe', 'en', 3, 'xsd:integer', 'Jane', 'Smith', 'fr', 3.14, 'xsd:decimal']
Processing file: input8.json
Loaded JSON data:
{'persons': [{'firstname': 'John', 'lastname': 'Doe', 'lang': 'en', 'num': 3, 'dt': 'int'}, {'firstname': 'Jane', 'lastname': 'Smith', 'lang': 'fr', 'num': '3.14', 'dt': 'dec'}]}
Extracted keys and values:
['firstname', 'John', 'la

In [26]:

def findingrowscolumns(keys):
    start = ''
    ei = 0
    count = 0

    for i in range(len(keys)):
        if start == '':
            start = keys[i]
            si = 0
        elif keys[i] == start:
            end = keys[i-1]
            ei = i
            break
        else:
            ei = i

    j = ei + 1
    count = 1
    for i in range(len(keys)):
        if j >= len(keys):
            break
        for k in range(ei):
            if not comparekeys(keys[k], keys[k+ei]):
                print("Keys are not of same length")
        j = j + ei
        count = count + 1

    if j == ei + 1:
        return count, ei + 1
    elif ei == 0:
        return 0, 0
    else:
        return count, ei

def comparekeys(w1, w2):
    return w1 == w2

def seperatekeyvalues(stackelements):
    keys = []
    values = []

    for i in range(len(stackelements)):
        if i % 2 == 0:
            keys.append(stackelements[i])
        else:
            values.append(stackelements[i])
    return keys, values

Number of rows: 2
Number of columns: 5
Keys: ['firstname', 'lastname', 'lang', 'num', 'dt', 'firstname', 'lastname', 'lang', 'num', 'dt']
Values: ['John', 'Doe', 'en', 3, '', 'Jane', 'Smith', 'fr', '3.14', '']


In [22]:
def writingintoFile(fileName, rows, column, keys, values):
    # Reshape the values into a 2D array
    a = np.array(values).reshape(rows, column)
    
    # Convert the array into a DataFrame using the keys as column headers
    DF = pd.DataFrame(a, columns=keys[:column])
    
    # Save the DataFrame as a CSV file
    DF.to_csv(fileName, index=False)